In [4]:
from abc import ABCMeta, abstractmethod
from collections import defaultdict
from typing import *

from mousse import Accessor, Dataclass
from mousse.data import DataMetaclass

In [22]:
class Listener(metaclass=ABCMeta):
    @abstractmethod
    def on_change(self, key: str, val: Any, mediator: "Mediator" = None):
        pass

    
class MediatorItemAccess(Accessor):    
    def __set__(self, obj: "Mediator", value: Any):
        print(obj)
        obj.notify(self.key)
        super().__set__(obj, value)


class MediatorMetaclass(DataMetaclass):
    def __new__(cls, name: str, bases: Tuple[type, ...], data: Dict[str, Any], accessor: Type[Accessor] = MediatorItemAccess):
        return super().__new__(cls, name, bases, data, accessor=accessor)         
    
    
class Mediator(metaclass=MediatorMetaclass):
    def __new__(cls):
        instance = super().__new__(cls)
        instance.listeners: Dict[str, List[Listener]] = defaultdict(list)
        return instance
    
    def build(self, *args, **kwargs):
        pass
        
    def notify(self, key: str):
        unique_listeners = set()
        for listener in self.listeners[key]:
            unique_listeners.add(listener)
            
        for listener in self.listeners[""]:
            unique_listeners.add(listener)
        
        val = getattr(self, key)
        for listener in unique_listeners:
            listener.on_change(key, val, mediator=self)
            
    def add(self, listener: "Listener", *keys: List[str]):
        if len(keys) == 0:
            keys += ("",)
            
        for key in keys:
            self.listeners[key].append(listener)

<class '__main__.MediatorItemAccess'>
{'__module__': '__main__', '__qualname__': 'Mediator', '__new__': <function Mediator.__new__ at 0x10930c8b0>, 'build': <function Mediator.build at 0x10930c940>, 'notify': <function Mediator.notify at 0x107443ee0>, 'add': <function Mediator.add at 0x1092dcc10>, '__classcell__': <cell at 0x1084500a0: empty>, '__init__': <function DataMetaclass.__new__.<locals>.__init__ at 0x1092dc670>, '__copy__': <function DataMetaclass.__new__.<locals>.__copy__ at 0x1092dc3a0>, '__deepcopy__': <function DataMetaclass.__new__.<locals>.__deepcopy__ at 0x1092dc1f0>, '__iter__': <function DataMetaclass.__new__.<locals>.__iter__ at 0x1092dc040>, '__repr__': <function DataMetaclass.__new__.<locals>.__repr__ at 0x1092dcf70>}


In [23]:
class ConcreteMediator(Mediator):
    name: str = "hello"
    age: int = -1

<class '__main__.MediatorItemAccess'>
{'__module__': '__main__', '__qualname__': 'ConcreteMediator', 'name': <__main__.MediatorItemAccess object at 0x1084b8100>, 'age': <__main__.MediatorItemAccess object at 0x108ae2b50>, '__init__': <function DataMetaclass.__new__.<locals>.__init__ at 0x107e35a60>, '__copy__': <function DataMetaclass.__new__.<locals>.__copy__ at 0x107e35b80>, '__deepcopy__': <function DataMetaclass.__new__.<locals>.__deepcopy__ at 0x109344040>, '__iter__': <function DataMetaclass.__new__.<locals>.__iter__ at 0x1093440d0>, '__repr__': <function DataMetaclass.__new__.<locals>.__repr__ at 0x109344160>}


In [24]:
mediator = ConcreteMediator()

class ConcreteListener(Listener):
    def on_change(self, key: str, val: Any, **kwargs):
        print("here", key, val)
        
listener = ConcreteListener()
mediator.add(listener, "age")

In [25]:
mediator1 = ConcreteMediator()
mediator2 = ConcreteMediator()

In [26]:
mediator.age = 1

ConcreteMediator(name="hello", age=-1)
here age -1


In [20]:
mediator.listeners

defaultdict(list, {'age': [<__main__.ConcreteListener at 0x1084adca0>]})

In [11]:
type(mediator.age)

int

In [80]:
ConcreteListener.a